In [15]:
import pandas as pd
import numpy as np
import xgboost as xgb
# from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pandas.tseries.holiday import *
cal = USFederalHolidayCalendar()

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
np.random.seed(422)
t = pd.read_csv("train.csv")


/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
train.drop(["Report Number","Local Case Number","Agency Name","Off-Road Description","Municipality","Circumstance","Vehicle ID","Driverless Vehicle", "Latitude", "Longitude","Municipality","Person ID","Vehicle Going Dir", "Vehicle Continuing Dir", "Vehicle Make", "Vehicle Model", "Drivers License State", "Road Name","Non-Motorist Substance Abuse","Cross-Street Name"], axis=1, inplace=True)

train["Crash Date/Time"] = pd.to_datetime(train["Crash Date/Time"])

train["hour"] = train["Crash Date/Time"].dt.hour
train["dayofweek"] = train["Crash Date/Time"].dt.dayofweek

holidays = cal.holidays(start=train['Crash Date/Time'].dt.date.min(), end=train['Crash Date/Time'].dt.date.max())
train['holiday'] = train['Crash Date/Time'].dt.date.astype('datetime64').isin(holidays).astype('uint8')

train.drop("Crash Date/Time", axis=1, inplace=True)

train['Location'] = train['Location'].str.replace(',',' ')
train[['l1', 'l2']] = train.Location.str.split(' ', expand = True ).astype(float)
train.drop(columns = ['Location'], inplace = True)

train["Parked Vehicle"] = train["Parked Vehicle"].astype("category").cat.codes
train["ACRS Report Type"] = train["ACRS Report Type"].astype("category").cat.codes

train["Vehicle First Impact Location"].fillna("UNKNOWN", inplace=True)
train["Vehicle Second Impact Location"].fillna("UNKNOWN", inplace=True)
train["Vehicle First Impact Location"] = train["Vehicle First Impact Location"].astype("category").cat.codes
train["Vehicle Second Impact Location"] = train["Vehicle Second Impact Location"].astype("category").cat.codes

train["Vehicle Movement"].fillna("UNKNOWN", inplace=True)
train["Vehicle Movement"] = train["Vehicle Movement"].astype("category").cat.codes

train["Equipment Problems"].fillna("UNKNOWN", inplace=True)
train["Equipment Problems"][3] = "NOT STREPPED RIGHT"
train["Equipment Problems"][8] = "SIZE/TYPE IMPROPER"
train["Equipment Problems"] = train["Equipment Problems"].astype("category").cat.codes

train["Vehicle Body Type"].fillna("UNKNOWN", inplace=True)
train["Vehicle Body Type"] = train["Vehicle Body Type"].astype("category").cat.codes

train["Vehicle Damage Extent"].fillna("UNKNOWN", inplace=True)
train["Vehicle Damage Extent"] = train["Vehicle Damage Extent"].astype("category").cat.codes

train["Cross-Street Type"].fillna("UNKNOWN", inplace=True)
train["Cross-Street Type"] = train["Cross-Street Type"].astype("category").cat.codes

train["Related Non-Motorist"].fillna("NIL", inplace=True)
train["Related Non-Motorist"] = train["Related Non-Motorist"].astype("category").cat.codes

train["Route Type"].fillna("UNKNOWN", inplace=True)
train["Route Type"] = train["Route Type"].astype("category").cat.codes

train["Injury Severity"] = train["Injury Severity"].astype("category").cat.codes

train["Traffic Control"].fillna("UNKNOWN", inplace=True)
train["Traffic Control"] = train["Traffic Control"].astype("category").cat.codes

train["Light"].fillna("UNKNOWN", inplace=True)
train["Light"] = train["Light"].astype("category").cat.codes

train["Surface Condition"].fillna("UNKNOWN", inplace=True)
train["Surface Condition"] = train["Surface Condition"].astype("category").cat.codes

train["Collision Type"].fillna("UNKNOWN", inplace=True)
train["Collision Type"] = train["Collision Type"].astype("category").cat.codes

train["Weather"].fillna("UNKNOWN", inplace=True)
train["Weather"] = train["Weather"].astype("category").cat.codes

train["Driver Substance Abuse"].fillna("UNKNOWN", inplace=True)
train["Driver Substance Abuse"] = train["Driver Substance Abuse"].astype("category").cat.codes

train.drop("Vehicle Year", axis=1,inplace=True)



<ipython-input-16-81cc5a28d551>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["Equipment Problems"][3] = "NOT STREPPED RIGHT"
<ipython-input-16-81cc5a28d551>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["Equipment Problems"][8] = "SIZE/TYPE IMPROPER"


In [17]:
keys = test["Id"]
test.drop("Id",axis=1,inplace=True)

test.drop(["Report Number","Local Case Number","Agency Name","Off-Road Description","Municipality","Circumstance","Vehicle ID","Driverless Vehicle", "Latitude", "Longitude","Municipality","Person ID","Vehicle Going Dir", "Vehicle Continuing Dir", "Vehicle Make", "Vehicle Model", "Drivers License State", "Road Name","Non-Motorist Substance Abuse","Cross-Street Name"], axis=1, inplace=True)

test["Crash Date/Time"] = pd.to_datetime(test["Crash Date/Time"])

test["hour"] = test["Crash Date/Time"].dt.hour
test["dayofweek"] = test["Crash Date/Time"].dt.dayofweek

holidays = cal.holidays(start=test['Crash Date/Time'].dt.date.min(), end=test['Crash Date/Time'].dt.date.max())
test['holiday'] = test['Crash Date/Time'].dt.date.astype('datetime64').isin(holidays).astype('uint8')

test.drop("Crash Date/Time", axis=1, inplace=True)

test['Location'] = test['Location'].str.replace(',',' ')
test[['l1', 'l2']] = test.Location.str.split(' ', expand = True ).astype(float)
test.drop(columns = ['Location'], inplace = True)

test["Parked Vehicle"] = test["Parked Vehicle"].astype("category").cat.codes
test["ACRS Report Type"] = test["ACRS Report Type"].astype("category").cat.codes

test["Vehicle First Impact Location"].fillna("UNKNOWN", inplace=True)
test["Vehicle Second Impact Location"].fillna("UNKNOWN", inplace=True)
test["Vehicle First Impact Location"] = test["Vehicle First Impact Location"].astype("category").cat.codes
test["Vehicle Second Impact Location"] = test["Vehicle Second Impact Location"].astype("category").cat.codes

test["Vehicle Movement"].fillna("UNKNOWN", inplace=True)
test["Vehicle Movement"] = test["Vehicle Movement"].astype("category").cat.codes

test["Equipment Problems"].fillna("UNKNOWN", inplace=True)
test["Equipment Problems"] = test["Equipment Problems"].astype("category").cat.codes

test["Vehicle Body Type"].fillna("UNKNOWN", inplace=True)
test["Vehicle Body Type"] = test["Vehicle Body Type"].astype("category").cat.codes

test["Vehicle Damage Extent"].fillna("UNKNOWN", inplace=True)
test["Vehicle Damage Extent"] = test["Vehicle Damage Extent"].astype("category").cat.codes

test["Cross-Street Type"].fillna("UNKNOWN", inplace=True)
test["Cross-Street Type"] = test["Cross-Street Type"].astype("category").cat.codes

test["Related Non-Motorist"].fillna("NIL", inplace=True)
test["Related Non-Motorist"] = test["Related Non-Motorist"].astype("category").cat.codes

test["Route Type"].fillna("UNKNOWN", inplace=True)
test["Route Type"] = test["Route Type"].astype("category").cat.codes

test["Injury Severity"] = test["Injury Severity"].astype("category").cat.codes

test["Traffic Control"].fillna("UNKNOWN", inplace=True)
test["Traffic Control"] = test["Traffic Control"].astype("category").cat.codes

test["Light"].fillna("UNKNOWN", inplace=True)
test["Light"] = test["Light"].astype("category").cat.codes

test["Surface Condition"].fillna("UNKNOWN", inplace=True)
test["Surface Condition"] = test["Surface Condition"].astype("category").cat.codes

test["Collision Type"].fillna("UNKNOWN", inplace=True)
test["Collision Type"] = test["Collision Type"].astype("category").cat.codes

test["Weather"].fillna("UNKNOWN", inplace=True)
test["Weather"] = test["Weather"].astype("category").cat.codes

test["Driver Substance Abuse"].fillna("UNKNOWN", inplace=True)
test["Driver Substance Abuse"] = test["Driver Substance Abuse"].astype("category").cat.codes

test.drop("Vehicle Year", axis=1,inplace=True)

# test_target = test["Fault"]
# test.drop("Fault", axis=1, inplace=True)

In [18]:
# reg = GradientBoostingClassifier(n_estimators = 500, max_depth=6)
# reg.fit(train, train_target)

# preds = reg.predict(test)
# submission = pd.DataFrame({'Id': keys, 'Fault':preds})
# submission.to_csv('sample_submission.csv',index=False)

In [19]:
"""
dtrain = xgb.DMatrix(train, label=train_target)
# param = {'max_depth': 8, 'eta': 0.06, 'objective': 'binary:hinge'}
param = {'booster':'dart', 'max_depth':6,'eta': 0.06, 'objective': 'binary:hinge', 'rate_drop':0.01, 'sample_type':'weighted'}
dtest = xgb.DMatrix(test)
bst = xgb.train(param, dtrain, 1000)
preds = np.array(bst.predict(dtest),dtype=int)
submission = pd.DataFrame({'Id': keys, 'Fault':preds})
submission.to_csv('sample_submission.csv',index=False)
"""

"\ndtrain = xgb.DMatrix(train, label=train_target)\n# param = {'max_depth': 8, 'eta': 0.06, 'objective': 'binary:hinge'}\nparam = {'booster':'dart', 'max_depth':6,'eta': 0.06, 'objective': 'binary:hinge', 'rate_drop':0.01, 'sample_type':'weighted'}\ndtest = xgb.DMatrix(test)\nbst = xgb.train(param, dtrain, 1000)\npreds = np.array(bst.predict(dtest),dtype=int)\nsubmission = pd.DataFrame({'Id': keys, 'Fault':preds})\nsubmission.to_csv('sample_submission.csv',index=False)\n"

In [20]:
import lightgbm as lgb
# dtrain = lgb.Dataset(train, label=train_target)
# param = {'verbose':-1,'task':'train','boosting':'dart', 'max_depth':6,'learning_rate': 0.06, 'objective': 'binary', 'rate_drop':0.02,'num_leaves':1024,'metric':'binary_logloss'}
# bst = lgb.train(param, dtrain, 1000)
# preds = bst.predict(test)
# preds = preds.round(0)
# preds = preds.astype(int)
# submission = pd.DataFrame({'Id': keys, 'Fault':preds})
# submission.to_csv('sample_submission.csv',index=False)
# accuracy_score(test_target, preds)
train, val = train_test_split(train, test_size=0.2)
train_target = train["Fault"]
train.drop("Fault", axis=1, inplace=True)
y_val = val['Fault']
X_val = val.drop('Fault',axis=1)

In [44]:
d_train=lgb.Dataset(train, label=train_target)#Specifying the parameter
# d_train = lgb.Dataset(X,label=y)
params={}
params['learning_rate']=0.1
params['boosting_type']='dart' 
params['objective']='binary'
params['metric']='binary_logloss' 
params['drop_rate']=0.29
params['max_drop']=100
params['num_iterations']=5000
clf=lgb.train(params,d_train)

y_pred=clf.predict(X_val)
y_pred=y_pred.round(0)
y_pred=y_pred.astype(int)
print(accuracy_score(y_val,y_pred))

/home/pranav/.local/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 22621, number of negative: 18571
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.167440 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 772
[LightGBM] [Info] Number of data points in the train set: 41192, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.549160 -> initscore=0.197277
[LightGBM] [Info] Start training from score 0.197277
0.8435618566711983


In [22]:
X_test = np.array(test)
y_pred=clf.predict(X_test)
y_pred=y_pred.round(0)
y_pred=y_pred.astype(int)
print(np.sum(y_pred))
data = {'Id':keys,'Fault':y_pred}
ans = pd.DataFrame(data)
ans.to_csv('subn.csv',index=False)
ans

42496


,Id,Fault
0,0,1
1,1,0
2,2,0
3,3,1
4,4,1
...,...,...
77230,77230,0
77231,77231,1
77232,77232,0
77233,77233,1
